In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [920 kB]
Get:14 http:

In [28]:
from pyspark import SparkFiles
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

# Initialize SparkSession
spark = SparkSession.builder.appName("Load and Separate Columns").getOrCreate()

# Define the URL of the data
url = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user"  # Replace with the actual URL

# Add the file to SparkFiles
spark.sparkContext.addFile(url)

# Read the data from SparkFiles as a single column
user = spark.read.text("file://" + SparkFiles.get("u.user"))  # Replace with the actual filename

# Split the column into separate columns
split_col = F.split(user['value'], '\|')
user = user.withColumn('user_id', split_col.getItem(0)) \
    .withColumn('age', split_col.getItem(1)) \
    .withColumn('gender', split_col.getItem(2)) \
    .withColumn('occupation', split_col.getItem(3)) \
    .withColumn('zip_code', split_col.getItem(4)) \
    .drop('value')

1. See the first 25 entries

In [29]:
user.show(25)

+-------+---+------+-------------+--------+
|user_id|age|gender|   occupation|zip_code|
+-------+---+------+-------------+--------+
|user_id|age|gender|   occupation|zip_code|
|      1| 24|     M|   technician|   85711|
|      2| 53|     F|        other|   94043|
|      3| 23|     M|       writer|   32067|
|      4| 24|     M|   technician|   43537|
|      5| 33|     F|        other|   15213|
|      6| 42|     M|    executive|   98101|
|      7| 57|     M|administrator|   91344|
|      8| 36|     M|administrator|   05201|
|      9| 29|     M|      student|   01002|
|     10| 53|     M|       lawyer|   90703|
|     11| 39|     F|        other|   30329|
|     12| 28|     F|        other|   06405|
|     13| 47|     M|     educator|   29206|
|     14| 45|     M|    scientist|   55106|
|     15| 49|     F|     educator|   97301|
|     16| 21|     M|entertainment|   10309|
|     17| 30|     M|   programmer|   06355|
|     18| 35|     F|        other|   37212|
|     19| 40|     M|    libraria

2. See the last 10 entries

In [30]:
user.tail(10)

[Row(user_id='934', age='61', gender='M', occupation='engineer', zip_code='22902'),
 Row(user_id='935', age='42', gender='M', occupation='doctor', zip_code='66221'),
 Row(user_id='936', age='24', gender='M', occupation='other', zip_code='32789'),
 Row(user_id='937', age='48', gender='M', occupation='educator', zip_code='98072'),
 Row(user_id='938', age='38', gender='F', occupation='technician', zip_code='55038'),
 Row(user_id='939', age='26', gender='F', occupation='student', zip_code='33319'),
 Row(user_id='940', age='32', gender='M', occupation='administrator', zip_code='02215'),
 Row(user_id='941', age='20', gender='M', occupation='student', zip_code='97229'),
 Row(user_id='942', age='48', gender='F', occupation='librarian', zip_code='78209'),
 Row(user_id='943', age='22', gender='M', occupation='student', zip_code='77841')]

3. What is the number of observations in the dataset?

In [31]:
user.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- zip_code: string (nullable = true)



In [32]:
user.count()

944

4. What is the number of columns in the dataset?

In [33]:
len(user.columns)

5

5. Print the name of all the columns.

In [34]:
user.columns

['user_id', 'age', 'gender', 'occupation', 'zip_code']

6. How is the dataset indexed?

In [35]:
# By default, Spark DataFrames are not indexed in the same way as Pandas DataFrames.
# Spark DataFrames are distributed collections of data, and they don't have an explicit index.

from pyspark.sql.functions import monotonically_increasing_id
df = user.withColumn("id", monotonically_increasing_id())
df.show(5)

+-------+---+------+----------+--------+---+
|user_id|age|gender|occupation|zip_code| id|
+-------+---+------+----------+--------+---+
|user_id|age|gender|occupation|zip_code|  0|
|      1| 24|     M|technician|   85711|  1|
|      2| 53|     F|     other|   94043|  2|
|      3| 23|     M|    writer|   32067|  3|
|      4| 24|     M|technician|   43537|  4|
+-------+---+------+----------+--------+---+
only showing top 5 rows



7. What is the data type of each column?

In [36]:
user.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- zip_code: string (nullable = true)



8.  Print only the occupation column

In [37]:
user.select("occupation").show()

+-------------+
|   occupation|
+-------------+
|   occupation|
|   technician|
|        other|
|       writer|
|   technician|
|        other|
|    executive|
|administrator|
|administrator|
|      student|
|       lawyer|
|        other|
|        other|
|     educator|
|    scientist|
|     educator|
|entertainment|
|   programmer|
|        other|
|    librarian|
+-------------+
only showing top 20 rows



9. How many different occupations are in this dataset?

In [39]:
user.select("occupation").distinct().count()

22

10. What is the most frequent occupation?

In [40]:
user.groupBy("occupation").count().orderBy("count", ascending =False).show(1)

+----------+-----+
|occupation|count|
+----------+-----+
|   student|  196|
+----------+-----+
only showing top 1 row



11. Summarize the DataFrame.

In [41]:
user.describe().show()

+-------+-----------------+-----------------+------+-------------+------------------+
|summary|          user_id|              age|gender|   occupation|          zip_code|
+-------+-----------------+-----------------+------+-------------+------------------+
|  count|              944|              944|   944|          944|               944|
|   mean|            472.0|34.05196182396607|  NULL|         NULL| 50868.78810810811|
| stddev|272.3649512449549|12.19273973305903|  NULL|         NULL|30891.373254138176|
|    min|                1|               10|     F|administrator|             00000|
|    max|          user_id|              age|gender|       writer|          zip_code|
+-------+-----------------+-----------------+------+-------------+------------------+



12 Summarize all the columns

In [42]:
user.select("user_id","age","gender","occupation","zip_code").describe().show()

+-------+-----------------+-----------------+------+-------------+------------------+
|summary|          user_id|              age|gender|   occupation|          zip_code|
+-------+-----------------+-----------------+------+-------------+------------------+
|  count|              944|              944|   944|          944|               944|
|   mean|            472.0|34.05196182396607|  NULL|         NULL| 50868.78810810811|
| stddev|272.3649512449549|12.19273973305903|  NULL|         NULL|30891.373254138176|
|    min|                1|               10|     F|administrator|             00000|
|    max|          user_id|              age|gender|       writer|          zip_code|
+-------+-----------------+-----------------+------+-------------+------------------+



13 Summarize only the occupation column

In [43]:
user.select("occupation").describe().show()

+-------+-------------+
|summary|   occupation|
+-------+-------------+
|  count|          944|
|   mean|         NULL|
| stddev|         NULL|
|    min|administrator|
|    max|       writer|
+-------+-------------+



14. What is the mean age of users?

In [44]:
user.select("age").describe().show()

+-------+-----------------+
|summary|              age|
+-------+-----------------+
|  count|              944|
|   mean|34.05196182396607|
| stddev|12.19273973305903|
|    min|               10|
|    max|              age|
+-------+-----------------+



15. What is the age with least occurrence?

In [45]:
user.groupBy("age").count().orderBy("count", ascending =True).show(1)

+---+-----+
|age|count|
+---+-----+
| 11|    1|
+---+-----+
only showing top 1 row

